In [3]:
import numpy as np

In [4]:
#the log odds of one amino acid being substituted by another
blosum50 = np.loadtxt("blosum50.txt", dtype = 'i')

In [5]:
#map the characters with their indices
blosum_index_map = dict(zip(['A','R','N','D','C','Q','E','G','H','I','L','K',
                'M','F','P','S','T','W','Y','V'],
                  [i for i in range(len(blosum50))]))

#given the bases to consider swapping, get the blosum log probability score of swapping them
#by looking up the corresponding index of the chars and then accessing that index of the matrix 
def get_blosum_score(char1, char2):
    i = blosum_index_map[char1]
    j = blosum_index_map[char2]
    return blosum50[i][j]

In [6]:
from termcolor import colored

In [7]:
#assumes that the blosum50 field is accessible in its environment
def needleman_wunsch(str2, str1):
    
    #our penalty score
    d = -8
    
    #str1 is the chars on the "left" of the matrix
    #str2 is the chars along the "top" of the matrix
    
    #initialize an empty matrix
    height = len(str1) + 1
    width = len(str2) + 1
    
    #explicitly object data type so that we can store tuples
    matrix = np.zeros((height, width), dtype = "object")
    
    #initialization step, mark the cells at the sides of the matrix
    #as incrementing by the penalty, and mark the "arrow" directions all as north/west
    count = 0
    for i in range(width):
        matrix[0][i] = (count,"w")
        count += d
    count = 0
    for i in range(height):
        matrix[i][0] = (count, "n")
        count +=d
    
    for i in range(1, height):
        
        for j in range(1, width):
            
            char1 = str1[i - 1]
            char2 = str2[j - 1]
            score = get_blosum_score(char2, char1)
            
            #direct implementation of the dynamic programming step
            matrix[i][j] = max([
                #each cell contains a tuple of score and direction so extract the score
                (matrix[i-1][j-1][0] + score,"nw"),
                (matrix[i-1][j][0] + d,"n"),
                (matrix[i][j-1][0] + d,"w")
            ], key = lambda tup: tup[0])
        
    #F matrix has been generated, now need to perform backwards algorithm
    #start at the bottom right of the matrix
    i = height - 1
    j = width - 1
    
    #path of cell transitions we will build up
    path = []
    
    #the 2 string substitution visualizations we will build up
    str1sub = ""
    str2sub = ""
    
    while True:
        
        #end condition, we have retraced the path
        if i == 0 and j == 0:
            break
        
        #the direction we need to go in
        dr = matrix[i][j][1]
        
        #chars stayed the same
        if dr == "nw":
            path.append((i,j))
            str1sub += str1[i - 1]
            str2sub += str2[j - 1]
            i -= 1
            j -= 1
            continue
        
        #eliminate char from str2
        if dr == "n":
            path.append((i,j))
            str1sub += str1[i - 1]
            str2sub += '-'
            i -= 1
            continue
        
        #eliminate char from str1
        if dr == "w":
            path.append((i,j))
            str1sub += '-'
            str2sub += str2[j - 1]
            j -= 1
    
    path.append((0,0))
    
    print("\nIndices of path found starting from bottom right:\n\n" + str(path) + "\n")
    
    print("\nString comparison, dashes represent if a character was eliminated:\n\n")
    #reverse the strings as they have been built up backwards
    print(str2sub[::-1] + "\n")
    print(str1sub[::-1] + "\n\n")
    
    print("Solution visualization on F matrix:\n")
    
    for i in range(height):
        
        for j in range(width):
            
            if((i,j) in path):
                print(colored(matrix[i][j][0], "red"), end = '')
            else:
                print(matrix[i][j][0], end = '')
        
        #print a new line
        print()        

In [8]:
needleman_wunsch("HEAGAWGHEE", "PAWHEAE")


Indices of path found starting from bottom right:

[(7, 10), (6, 9), (5, 9), (4, 8), (3, 7), (3, 6), (2, 5), (1, 4), (1, 3), (0, 2), (0, 1), (0, 0)]


String comparison, dashes represent if a character was eliminated:


HEAGAWGHE-E

--P-AW-HEAE


Solution visualization on F matrix:

0-8-16-24-32-40-48-56-64-72-80
-8-2-9-17-25-33-41-49-57-65-73
-16-10-3-4-12-20-28-36-44-52-60
-24-18-11-6-7-15-5-13-21-29-37
-32-14-18-13-8-9-13-7-3-11-19
-40-22-8-16-16-9-12-15-73-5
-48-30-16-3-11-11-12-12-15-52
-56-38-24-11-6-12-14-15-12-91


In [9]:
needleman_wunsch("SALPQPTTPVSSFTSGSMLGRTDTALTNTYSAL", "PSPTMEAVTSVEASTASHPHSTSSYFATTYYHLY")


Indices of path found starting from bottom right:

[(34, 33), (33, 33), (32, 32), (31, 31), (30, 30), (29, 29), (28, 28), (27, 27), (26, 26), (25, 25), (24, 24), (23, 23), (22, 22), (21, 21), (20, 20), (19, 19), (18, 18), (17, 17), (16, 16), (15, 15), (14, 14), (13, 13), (13, 12), (12, 11), (11, 10), (10, 9), (9, 8), (8, 7), (7, 6), (6, 5), (5, 4), (4, 3), (3, 2), (2, 1), (1, 0), (0, 0)]


String comparison, dashes represent if a character was eliminated:


-SALPQPTTPVSSFTSGSMLGRTDTALTNTYSAL-

PSPTMEAVTSVEA-STASHPHSTSSYFATTYYHLY


Solution visualization on F matrix:

0-8-16-24-32-40-48-56-64-72-80-88-96-104-112-120-128-136-144-152-160-168-176-184-192-200-208-216-224-232-240-248-256-264
-8-1-9-17-14-22-30-38-46-54-62-70-78-86-94-102-110-118-126-134-142-150-158-166-174-182-190-198-206-214-222-230-238-246
-16-30-8-16-14-22-28-36-44-52-57-65-73-81-89-97-105-113-121-129-137-145-153-161-169-177-185-193-201-209-217-225-233
-24-11-4-42-6-4-12-20-26-34-42-50-58-66-74-82-90-98-106-114-122-130-1

In [10]:
#assumes that the blosum50 field is accessible in its environment
def smith_waterman(str2, str1):
    
    #our penalty score
    d = -8
    
    #str1 is the chars along the "left" of the matrix
    #str2 is the chars along the "right" of the matrix
    
    #initialize an empty matrix
    height = len(str1) + 1
    width = len(str2) + 1
    
    #explicitly object data type so that we can store tuples
    matrix = np.zeros((height, width), dtype = "object")
    
    #the cells at the edges are now marked as zeroes
    for i in range(width):
        matrix[0][i] = (0,"zero")
    for i in range(height):
        matrix[i][0] = (0, "zero")
    
    #keep track of the max seen 
    max_seen = 0
    #also keep track of its index
    max_index = (0,0)
    
    for i in range(1, height):
        
        for j in range(1, width):
            
            char1 = str1[i - 1]
            char2 = str2[j - 1]
            score = get_blosum_score(char2, char1)
            
            #direct implementation of the dynamic programming step
            matrix[i][j] = max([
                #the new case added for smith waterman
                (0, "zero"),
                #each cell contains a tuple of score and direction so extract the score
                (matrix[i-1][j-1][0] + score,"nw"),
                (matrix[i-1][j][0] + d,"n"),
                (matrix[i][j-1][0] + d,"w")
            ], key = lambda tup: tup[0])
            
            #keep track of the biggest cell value found as this
            #will now be our starting point
            if(matrix[i][j][0] > max_seen):
                max_seen = matrix[i][j][0]
                max_index = (i,j)
        
    #we are now starting at the index containing the max value 
    #instead of the bottom right of the matrix
    i = max_index[0]
    j = max_index[1]
    
    #path of cell transitions we will build up
    path = []
    
    #the 2 string substitution visualizations we will build up
    str1sub = ""
    str2sub = ""
    
    while True:
                
        #the direction we need to go in (or a zero)
        dr = matrix[i][j][1]
        
        print("at position:",i,j)
        print("dir is",dr)
        
        #termination condition, the path stops at 0
        if dr == "zero":
            break
        
        #chars stayed the same
        if dr == "nw":
            
            print("inside if")
            
            path.append((i,j))
            
            print("appended, now going to try the fancy string thing")
            
            str1sub += str1[i - 1]
            str2sub += str2[j - 1]
            
            print("It worked")
            
            i -= 1
            j -= 1
            continue
        
        #eliminate char from the string along the "top" of the matrix
        if dr == "n":
            path.append((i,j))
            str1sub += str1[i - 1]
            str2sub += '-'
            i -= 1
            continue
        
        #eliminate char from the string along the "left" of the matrix
        if dr == "w":
            path.append((i,j))
            str1sub += '-'
            str2sub += str2[j - 1]
            j -= 1
    
    #we finished at indices i,j therefore it must be a 0 and the final
    #node in the path, so add it to the path 
    path.append((i,j))
    
    print("\nIndices of path found starting from maximum:\n\n" + str(path) + "\n")
    
    print("\nLocal match substituted strings, dashes represent if a character was eliminated:\n")
    
    #reverse the strings as they have been built up backwards
    print(str2sub[::-1] + "\n")
    print(str1sub[::-1] + "\n\n")
    
    print("Solution visualization on F matrix:\n")
    
    for i in range(height):
        
        for j in range(width):
            
            if((i,j) in path):
                print(colored(matrix[i][j][0], "red")," ", end = '')
            else:
                print(matrix[i][j][0]," ", end = '')
        
        #print a new line
        print()        

In [11]:
smith_waterman("HEAGAWGHEE", "PAWHEAE")

at position: 5 9
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 4 8
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 3 7
dir is w
at position: 3 6
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 2 5
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 1 4
dir is zero

Indices of path found starting from maximum:

[(5, 9), (4, 8), (3, 7), (3, 6), (2, 5), (1, 4)]


Local match substituted strings, dashes represent if a character was eliminated:

AWGHE

AW-HE


Solution visualization on F matrix:

0  0  0  0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  0  0  0  
0  0  0  5  0  5  0  0  0  0  0  
0  0  0  0  2  0  20  12  4  0  0  
0  10  2  0  0  0  12  18  22  14  6  
0  2  16  8  0  0  4  10  18  28  20  
0  0  8  21  13  5  0  4  10  20  27  
0  0  6  13  18  12  4  0  4  16  26  


In [12]:
smith_waterman(
    "MQNSHSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRY",
    "TDDECHSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRI"
)

at position: 46 45
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 45 44
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 44 43
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 43 42
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 42 41
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 41 40
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 40 39
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 39 38
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 38 37
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at position: 37 36
dir is nw
inside if
appended, now going to try the fancy string thing
It worked
at positio

0  0  0  0  3  7  3  3  0  0  0  1  0  0  0  0  9  19  33  48  66  79  97  113  126  139  131  123  115  107  99  91  83  75  67  59  51  43  35  27  19  11  14  22  22  14  6  0  0  2  9  1  1  0  2  4  0  0  0  0  0  
0  0  1  0  0  3  6  0  0  0  1  0  0  0  0  0  1  11  25  40  58  71  89  105  118  131  146  138  130  122  114  106  98  90  82  74  66  58  50  42  34  26  18  14  29  21  13  7  0  0  1  6  0  0  0  5  0  0  0  7  0  
0  0  7  1  0  1  3  4  0  0  7  0  0  0  0  0  0  3  17  32  50  63  81  97  110  123  138  153  145  137  129  121  113  105  97  89  81  73  65  57  49  41  33  25  21  26  19  20  12  4  0  0  3  0  0  2  2  0  0  1  6  
0  0  2  7  1  0  1  1  1  0  2  4  0  0  0  0  0  0  9  24  42  55  73  89  102  115  130  145  159  151  143  135  127  119  111  103  95  87  79  71  63  55  47  39  31  23  23  21  17  12  4  0  0  0  0  6  0  0  0  3  0  
0  2  0  0  4  0  0  0  5  0  0  4  0  0  4  0  4  0  1  16  34  47  65  81  94  107  122  137  151  164 

## Question 3

In [13]:
import os

In [14]:
os.listdir()

['3CYASRFB114-Alignment.txt',
 'weeks 3-5.ipynb',
 '.ipynb_checkpoints',
 'O96791.fasta',
 'P63015.fasta',
 'blosum50.txt',
 'phaseLambda.fasta']

In [15]:
nuc_bases = ['A', 'T', 'C', 'G']

In [16]:
string = open("phaseLambda.fasta", 'r').read().replace('\n', '')

In [17]:
#the length of the phase lambda is 48502 so this is how large
#the HMM generated sequence will be
len(string)

48502

I found the most elegant way to model the HMM was using mutual recursion, where the 2 functions calling one another represents a change in state, and when n_remaining reaches 0 then the recursive stack that has been built up is returned (which will ultimately be a list of tuples of the string sections generated by each state, paired with a string stating the state that generated the section)

In [18]:
from functools import reduce

In [19]:
def run_HMM():
        
    #we want to generate CG rich regions, so start in the CG rich state
    tups = generate_CGrich_region(48502, [])
    
    #return the sections generated paired with strings stating the states that generated them,
    #paired with the whole concatenated string that the HMM produced
    return tups, str(reduce(lambda x,y: x + y, list(map(lambda x: x[0], tups))))
        
    #start in either state with equal chance
    #if np.random.choice([True, False], p = [0.5,0.5]):
    #    return generate_ATrich_region(48502, [])
    #else:
    #    return generate_CGrich_region(48502, [])

In [20]:
def generate_ATrich_region(n_remaining, accumulator):
    
    print("entered AT state, accumulator length:", len(accumulator))
    
    #emission probabilities for each of the nucleotide bases respectively
    #when in this state
    emission_probs = [0.2698, 0.3237, 0.2080, 0.1985]
    
    #a continuous section that will be built up whilst in this state
    section_str = ""
    
    #loop until we change into the other state by chance 
    while True:
        
        #whole recursive call finished and can be returned
        if n_remaining == 0:
            accumulator.append((section_str, "AT_rich"))
            print("Fully finished now in AT state")
            return accumulator
        
        section_str += np.random.choice(nuc_bases, p = emission_probs)
        n_remaining -= 1
        
        #on each iteration we may by chance finish in this state
        #and go into the other state
        if np.random.choice([True, False], p = [0.0002, 0.9998]):
            break
    
    print("finished in AT state, string length:",len(section_str))
    accumulator.append((section_str, "AT_rich"))
    return generate_CGrich_region(n_remaining, accumulator)

In [21]:
def generate_CGrich_region(n_remaining, accumulator):
    
    print("entered CG state, accumulator length:", len(accumulator))
    
    #emission probabilities for each of the nucleotide bases respectively
    #when in this state
    emission_probs = [0.2459, 0.2079, 0.2478, 0.2984]
    
    #a continuous section that will be built up whilst in this state
    section_str = ""
    
    #loop until we change into the other state by chance
    while True:
        
        #whole recursive call finished and can be returned
        if n_remaining == 0:
            accumulator.append((section_str, "CG_rich"))
            print("Fully finished now in CG state")
            return accumulator
    
        section_str += np.random.choice(nuc_bases, p = emission_probs)
        n_remaining -= 1
        
        #on each iteration we may by chance finish in this state
        #and go into the other state
        if np.random.choice([True, False], p = [0.0002, 0.9998]):
            break
    
    print("finished in CG state, string length:",len(section_str))
    accumulator.append((section_str, "CG_rich"))
    return generate_ATrich_region(n_remaining, accumulator)        

In [22]:
tups, string = run_HMM()

entered CG state, accumulator length: 0
finished in CG state, string length: 8766
entered AT state, accumulator length: 1
finished in AT state, string length: 128
entered CG state, accumulator length: 2
finished in CG state, string length: 2599
entered AT state, accumulator length: 3
finished in AT state, string length: 1179
entered CG state, accumulator length: 4
finished in CG state, string length: 4459
entered AT state, accumulator length: 5
finished in AT state, string length: 8644
entered CG state, accumulator length: 6
finished in CG state, string length: 3566
entered AT state, accumulator length: 7
finished in AT state, string length: 186
entered CG state, accumulator length: 8
finished in CG state, string length: 702
entered AT state, accumulator length: 9
finished in AT state, string length: 8262
entered CG state, accumulator length: 10
finished in CG state, string length: 9487
entered AT state, accumulator length: 11
Fully finished now in AT state


In [23]:
tups

[('AGCAGTGCACGTCGTTCCATTACCTCGCCGCTCTGTTCACCCAGGATCTCCAACAGCACAGTGAAGGGTCTCCCGACGCGAACGGGGTAGAGGTATCGCTCGGCGTTACCCCCGTGCGCTAAGAGAGGTATGACCTACTTTCCAAGAGTGAGCGAACAACTAGGGGAACATGGCGATTGGAGGCACAAGGCTATACCCCAGTTACTGACTGCTGGGGTCATTGCGATTGAGATGCGGACACCTTTAGTTTATACGTCCCACTGGAATCGATCGCAGAATAGAGGATGCAGCCGTTACGCGATGTGGCACTACACATCATGCACCATTACTAAGGTAGGCGCGCGGAGGGGCTATTGAACGGGAGCTTAGGATTAACGTTATCCTGTCCCTCCAAGACATCCAACACGCGCTACGCAGAGGGTCGAGATCTGGTCGCGATGACCTCCTTGACCGCGCGCCGAGGAACATAAAAGTGTCGTATACAATAAGCTCGGAAATTAGCGAAACGAGTGGGTCTCAACTGTGTTGGTCACTCACAAACCAGCCGCGCTTAGAGTCCAGCGATCTGGGATGGAGGGCGGCCCCAGCAGCCGGGCGGCGAACCAACGGTGTGGCGAAACATATAGTCGCATCTACGCGCATCTGAGCGGGGGACCTTCATTGCGGATACATTCACTAAACCGAGTCTGGGCGGTTGTAATGGCCTGGACCCCCAAGGTGGTAGAAGTCCGATAGAGGGAGTGATGCACTCCACTCATCTAGAGAAATGTGATGTTAGATGTACCTAACTCGAGACAGGCGATATAGTCAAGAGTTACGTGGAAGCTGTCTTCCCGCACATGGTGAATCGTCACACGTCGTTTCAGGGTCAAGGTTTAGGCGAGTAGTATTGCAGTAACACGAGTAGCACGTGCTTCTTAATCAACTCGCCAGTTAGAGAAGGTCAAACCTCGGTCTAAGCGAGAGCAGCACCGACGAATCCGGAGGGAACGTCGGC

In [26]:
sum(list(map(lambda x: len(x[0]), result)))

NameError: name 'result' is not defined

In [40]:
from scipy.special import logsumexp, logsumexp

Now lets match the states and nucleotide base letters with index digits so that we can place them into our A and B matrices

we will set AT_rich, CG_rich as 0 and 1 respecitvely

and A, T, C, G as 0, 1, 2 and 3 respectively:

In [48]:
#takes as input the sequence of observations, which is just one large string
def viterbi_decode(observations): #obersvation sequence is often denotes as Y
    
    #Aij is the probability of tranistioning from state si to state sj
    A = [[0.9998, 0.0002],[0.9997, 0.0003]]
    
    #Bij stores the probability of observing oj from state si
    
    # It makes sense to store as a list of dictionaries as we will
    # always be querying the inner "lists" (so dictionaries instead here) 
    # of B based on the nucleotide base letters and not indices
    B = [
        {'A': 0.2698, 'G': 0.3237, 'C': 0.2080, 'T': 0.1985}, 
        {'A': 0.2459, 'G': 0.2079, 'C': 0.2478, 'T': 0.2984}
        ]
    
    #all possible observations, (O)
    observation_space = ['A', 'G', 'C', 'T']
    
    #all possible states that can be entered (S)
    state_space = ["AT_rich", "CG_rich"]
    
    #equal chance of starting in either state (PI)
    initial_probs = [0.5,0.5]
    
    #T is the length of the observation sequence, so T = len(observations)
    T = len(observations)
    
    #K is the cardinality of the set of possible states that can be entered (S), so K = 2
    K = len(state_space)
    
    #N is the cardinality of the set of possible observations, so N = 4
    N = len(observation_space)
    
    #replace with whatever you want to be your data type here
    T1 = np.zeros((K, T), dtype = "int32")
    T2 = np.zeros((K, T), dtype = "int32")
    
    for i, _ in enumerate(state_space):
        T1[i, 1] = initial_probs[i] * B[i][observations[0]]
        T2[i, 1] = 0
    
    for j in range(2, T):
    
        for i in range(K):
            
            T1[i][j] = 
    

In [49]:
viterbi_decode("ACTGTAGCTAGCTACGATGCATGTCAGTGCATGCTACGATCG")

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0]]


In [24]:
from scipy.special import logsumexp

In [29]:
#takes as input the observed sequence string
def viterbi_v2(sequence_string):
    
    #nucleotide base symbol emission probabilities
    AT_emission_probs = {'A':0.2698,'T':0.3237,'C':0.2080,'G':0.1985}
    CG_emission_probs = {'A':0.2459,'T':0.2079,'C':0.2478,'G':0.2984}
    
    #state transition probabilities
    AT_stay_same = 0.9 #0.9998
    AT_2_CG = 0.1 #0.0002
    CG_stay_same = 0.9 #0.9997
    CG_2_AT = 0.1 #0.0003
    
    #how I will model the "rows" in the viterbi algorithm diagram, using 2 seperate lists
    #i.e. the cumulative probabilities for being in either state 
    #and generating a letter at some point in the sequence
    
    #lets model the cumulative probability of the start state as 0
    #and the probability of transitioning from it to either the AT rich 
    #or CG rich state as 0.5 for both 
    AT_cumulative_probs = [(1,"end")]
    CG_cumulative_probs = [(1,"end")]
    
    #second part of the tuple is either end, from_same or from_diff, 
    #i.e. we likely came from the same state or from a different state
    
    #if(np.random.choice([True, False], p = [0.5,0.5])):
    #    AT_cumulative_probs.append((np.logaddexp(AT_stay_same, AT_emission_probs[sequence_string[0]]), "end"))
    #    CG_cumulative_probs.append(np.logaddexp(AT_2))
   # else:
    #    AT_cumulative_probs.append()
    #    CG_cumulative_probs.append()
    
    #the "forward" part of the viterbi algorithm
    for char in sequence_string[1:]:
        
        #the cumulative probabilities in the predecessor AT/CG state nodes
        last_AT_val = AT_cumulative_probs[-1][0]
        last_CG_val = CG_cumulative_probs[-1][0]
        
        AT_max_val, AT_state_came_from = max([(last_CG_val + np.log(CG_2_AT),"from_other"),(last_AT_val + np.log(AT_stay_same),"from_same")], key = lambda tup: tup[0])
        CG_max_val, CG_state_came_from = max([(last_AT_val + np.log(AT_2_CG),"from_other"),(last_CG_val + np.log(CG_stay_same),"from_same")], key = lambda tup: tup[0])
        
        AT_cumulative_probs.append((np.log(AT_emission_probs[char]) + AT_max_val, AT_state_came_from))
        CG_cumulative_probs.append((np.log(CG_emission_probs[char]) + CG_max_val, CG_state_came_from))
    
    return AT_cumulative_probs, CG_cumulative_probs

In [30]:
at_cumulative_probs, cg_cumulative_probs = viterbi_v2(string)

In [31]:
at_cumulative_probs[5000:]

[(-7412.523822433415, 'from_same'),
 (-7413.757121067123, 'from_same'),
 (-7415.479447761635, 'from_same'),
 (-7417.201774456147, 'from_same'),
 (-7418.76934006792, 'from_other'),
 (-7420.08402099423, 'from_other'),
 (-7421.091810076894, 'from_other'),
 (-7422.50724492776, 'from_same'),
 (-7424.182822642699, 'from_same'),
 (-7425.416121276407, 'from_same'),
 (-7426.649419910114, 'from_same'),
 (-7427.882718543822, 'from_same'),
 (-7429.298153394689, 'from_same'),
 (-7430.713588245555, 'from_same'),
 (-7432.435914940067, 'from_same'),
 (-7434.111492655005, 'from_same'),
 (-7435.344791288713, 'from_same'),
 (-7437.067117983225, 'from_same'),
 (-7438.482552834092, 'from_same'),
 (-7440.15813054903, 'from_same'),
 (-7441.573565399896, 'from_same'),
 (-7443.295892094408, 'from_same'),
 (-7444.529190728116, 'from_same'),
 (-7445.944625578983, 'from_same'),
 (-7447.666952273495, 'from_same'),
 (-7449.342529988433, 'from_same'),
 (-7450.757964839299, 'from_same'),
 (-7452.480291533811, 'from_s

In [35]:
cg_cumulative_probs[30000:]

[(-44360.69948768898, 'from_same'),
 (-44362.19998151442, 'from_same'),
 (-44363.51466244073, 'from_same'),
 (-44365.190721040504, 'from_same'),
 (-44366.86677964028, 'from_same'),
 (-44368.181460566586, 'from_same'),
 (-44369.68195439203, 'from_same'),
 (-44371.190145237444, 'from_same'),
 (-44372.69833608286, 'from_same'),
 (-44374.013017009165, 'from_same'),
 (-44375.51351083461, 'from_same'),
 (-44377.01400466005, 'from_same'),
 (-44378.690063259826, 'from_same'),
 (-44380.19055708527, 'from_same'),
 (-44381.69105091071, 'from_same'),
 (-44383.19154473615, 'from_same'),
 (-44384.692038561596, 'from_same'),
 (-44386.36809716137, 'from_same'),
 (-44387.68277808768, 'from_same'),
 (-44389.18327191312, 'from_same'),
 (-44390.68376573856, 'from_same'),
 (-44392.184259564005, 'from_same'),
 (-44393.86031816378, 'from_same'),
 (-44395.174999090086, 'from_same'),
 (-44396.67549291553, 'from_same'),
 (-44398.351551515305, 'from_same'),
 (-44399.85204534075, 'from_same'),
 (-44401.3525391661